## Simple Requests
Simple requests tests on each llms that will be used in the debate. 

In [ ]:
# Necessary imports
import os
from dotenv import load_dotenv

from openai import OpenAI
from pydantic import BaseModel, Field

load_dotenv(override=True)

### OpenAI
4 OpenAI models will be used:
- GPT 4.1 mini `gpt-4.1-mini`
- GPT 4.1 `gpt-4.1`
- GPT 5 mini `gpt-5-mini`
- GPT 5 `gpt-5`

In [ ]:
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! How are you today?"}
    ]
)

print(response.choices[0].message.content)

In [ ]:
# Test Structured output

class ResponseModel(BaseModel):
    answer: str = Field(..., description="The answer to the user's question.")
    confidence: float = Field(..., description="The confidence level of the answer.")
    other_details: str | None = Field(None, description="Any other relevant details.")

response = client.chat.completions.parse(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    response_format=ResponseModel
)

result = response.choices[0].message.parsed
result

### Gemini
2 Gemini models will be used:
- Gemini 2.5 Flash `gemini-2.5-flash`
- Gemini 2.5 Pro `gemini-2.5-pro`

In [ ]:
gemini_api_key = os.getenv("GEMINI_API_KEY")

client = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[
        {"role": "system", "content": "You are a trolling bot that only responds with sarcastic remarks."},
        {"role": "user", "content": "Hello! How are you today?"}
    ],
    max_completion_tokens=1024
)

print(response.choices[0].message.content)

In [ ]:
# Test Structured output with Gemini

class CityInfo(BaseModel):
    city: str = Field(..., description="Name of the city.")
    country: str = Field(..., description="Country where the city is located.")
    population: int = Field(..., description="Population of the city.")
    interesting_fact: str = Field(..., description="An interesting fact about the city.")

response = client.chat.completions.parse(
    model="gemini-2.5-flash",
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant."},
        {"role": "user", "content": "Provide information about Lubumbashi."}
    ],
    response_format=CityInfo,
)

result = response.choices[0].message.parsed
result

### Open Source
Open source models will be provided by Groq. We will use:
- Llama 3.1 8B `llama-3.1-8b-instant`
- Llama 3.3 70B `llama-3.3-70b-versatile`
- GPT-OSS 20B `openai/gpt-oss-20b`
- GPT-OSS 120B `openai/gpt-oss-120b`

In [ ]:
groq_api_key = os.getenv("GROQ_API_KEY")

client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

In [ ]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! How are you today?"}
    ]
)

print(response.choices[0].message.content)

In [ ]:
# Test Structured output with Groq
class BookInfo(BaseModel):
    title: str = Field(..., description="Title of the book.")
    author: str = Field(..., description="Author of the book.")
    published_year: int = Field(..., description="Year the book was published.")

response = client.chat.completions.parse(
    model="openai/gpt-oss-20b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Provide information about the book '1984' by George Orwell."}
    ],
    response_format=BookInfo,
)

result = response.choices[0].message.parsed
result